# Homework 1 &mdash; Part 2  (Due: 10/17/2022)

COEN 281, Fall 2022  
Professor Marwah

---

### Student name:
### Srudent ID:
### Student email:

The objective of this HW is to explore a real-life data set, and perfom exploratory data analysis (EDA) on it. Real data sets tend to be large, complex (a lot of heterogenous variables/features, many with unclear semantics), noisy (including outliers), incompplete (a lot of missing values), redundant (duplicated data) and generally messy. It is no surprise then that according to some sources 80% of a data scientist's time is spent collecting, cleaning and wrangling data.

This data set contains house sales data in the previous few years from CA. I have usually given [this data set](https://archive.ics.uci.edu/ml/datasets/census+income) for EDA when I have taught this class earlier. It is a nice data set, but it is old, curated and quite clean as far as real data sets are concerned. So I was quite excited when I discovered this publicly available, rich data set, and decided it would be a good exposure to real data sets, typically seen in the industry (although mostly the data sets remain private).

This data set was scrapped from a site, such as Zillow or Redfin by Alex Smola's team for a [course](https://c.d2l.ai/stanford-cs329p/) at Stanford. [This site](https://c.d2l.ai/stanford-cs329p/_static/notebooks/cs329p_notebook_eda.slides.html) shows some EDA on this data set. Please go through it. Some of the steps there are repeated here.

The main objectives of exploratory analyses is to get a sense of what the data looks like; how good is the data quality; explore relationships between variables. You may even have some preconceived notions of how different features look like; you can treat these as hypotheses and then try to figure out from the data if they really hold. 


Note: You can use either local runtimes to complete this assignment, or a hosted runtime on Colab. 


In [1]:
# You would need these libraries. Run the following run once to install if not already installed 
#!pip3 install pyarrow numpy pandas seaborn


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

Let's read the data first from the csv file. We put '%time' just to check how long it takes (otherwise it doesn't change anything)

In [3]:
%time data = pd.read_csv('house_sales.csv.gz', low_memory=False)

Wall time: 27.7 s


#### Aside
It takes about half a minute to read the file on my machine, which is not too bad since we only need to do it once. If it were a larger file (say 10x) then it would be a problem. There is another file format called [feather](https://arrow.apache.org/docs/python/feather.html) which saves dataframes (usually compressed) to files and is much faster in loading. (One drawback compared to csv is that it is a binary file, so you can't just less it to see its content or edit it ith a text editor.)
The same data is available as a feature file as well. Let's see how long it takes to load it. 

In [4]:
%time data2 = pd.read_feather('house_sales.ftr')

Wall time: 1.91 s


On my machine, reading the feature version of the data set was at least 10-15x faster. We will just continue with the csv file we read earlier. This was just to show you that faster alternatives exist in case your data set is larger (of course you need enough memory in your machine as well).

In [5]:
data.shape

(164944, 1789)

In [6]:
data.head()

,Id,Address,Sold Price,Sold On,Summary,Type,Year built,Heating,Cooling,Parking,...,Well Disclosure,remodeled,DOH2,SerialX,Full Baths,Tax Legal Lot Number,Tax Legal Block Number,Tax Legal Tract Number,Building Name,Zip
0,2080183300,"11205 Monterey,","$2,000,000",01/31/20,"11205 Monterey, San Martin, CA 95046 is a sing...",SingleFamily,No Data,No Data,No Data,0 spaces,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95046
1,20926300,"5281 Castle Rd,","$2,100,000",02/25/21,Spectacular Mountain and incredible L.A. City ...,SingleFamily,1951,Central,"Central Air, Dual","Driveway, Driveway - Brick",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91011
2,19595300,"3581 Butcher Dr,","$1,125,000",11/06/19,Eichler Style home! with Santa Clara High! in ...,SingleFamily,1954,Central Forced Air - Gas,Central AC,"Garage, Garage - Attached, Covered",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95051
3,300472200,"2021 N Milpitas Blvd,","$36,250,000",10/02/20,"2021 N Milpitas Blvd, Milpitas, CA 95035 is a ...",Apartment,1989,Other,No Data,"Mixed, Covered",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95035
4,2074492000,"LOT 4 Tool Box Spring Rd,","$140,000",10/19/20,Beautiful level lot dotted with pine trees ro...,VacantLand,No Data,No Data,No Data,0 spaces,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92561


The number of columns is especially large. While head only shows a few columns, several have NaNs. Let's look at the data types of the columns and then check how many missing values are in each column.

In [7]:

pd.set_option('display.max_rows', 1800)   # so it displays all the rows

data.dtypes

Id                                                                                                               int64
Address                                                                                                         object
Sold Price                                                                                                      object
Sold On                                                                                                         object
Summary                                                                                                         object
Type                                                                                                            object
Year built                                                                                                      object
Heating                                                                                                         object
Cooling                                         

In [9]:
# percentage missing values in each column
data.isnull().sum()/len(data)*100

Id                                                                                                              0.000000
Address                                                                                                         0.000000
Sold Price                                                                                                      0.051533
Sold On                                                                                                         0.000000
Summary                                                                                                         1.889732
Type                                                                                                            1.020953
Year built                                                                                                      1.003371
Heating                                                                                                         1.017315
Cooling                         

In [10]:
## number of columns that have > 90% missing
len(data.columns[ data.isnull().sum()/len(data)*100 > 90 ])

1690

In [11]:
## columns that have < 5% missing
data.columns[ data.isnull().sum()/len(data)*100 < 5 ]

Index(['Id', 'Address', 'Sold Price', 'Sold On', 'Summary', 'Type',
       'Year built', 'Heating', 'Cooling', 'Parking', 'Total spaces',
       'Garage spaces', 'Home type', 'Region', 'Zip'],
      dtype='object')

So most of the columns are very sparesely populated, making them unuseable. Let's drop columns where more than 20% of values are missing. 

In [12]:
data.drop(columns=data.columns[data.isnull().sum() > 0.2*len(data)], inplace=True)

In [13]:
data.shape

(164944, 29)

Now let's remove rows with missing values. We could try to impute these values (and that is a better way to handle a smaller number of missing values), but in this case since we have enough data we will simply remove them.  

In [14]:
data.dropna(inplace=True)

In [15]:
data.shape

(85720, 29)

In [16]:
data.head()

,Id,Address,Sold Price,Sold On,Summary,Type,Year built,Heating,Cooling,Parking,...,Elementary School Distance,High School,High School Score,High School Distance,Parking features,Lot size,Parcel number,Tax assessed value,Annual tax amount,Zip
1,20926300,"5281 Castle Rd,","$2,100,000",02/25/21,Spectacular Mountain and incredible L.A. City ...,SingleFamily,1951,Central,"Central Air, Dual","Driveway, Driveway - Brick",...,0.4,Crescenta Valley High School,9.0,1.2,"Driveway, Driveway - Brick","13,168 sqft",5870016003,"$1,829,308","$22,330",91011
2,19595300,"3581 Butcher Dr,","$1,125,000",11/06/19,Eichler Style home! with Santa Clara High! in ...,SingleFamily,1954,Central Forced Air - Gas,Central AC,"Garage, Garage - Attached, Covered",...,0.6,Santa Clara High School,6.0,0.8,"Garage, Garage - Attached, Covered","4,795 sqft",29022100,"$1,125,000","$13,472",95051
3,300472200,"2021 N Milpitas Blvd,","$36,250,000",10/02/20,"2021 N Milpitas Blvd, Milpitas, CA 95035 is a ...",Apartment,1989,Other,No Data,"Mixed, Covered",...,0.4,Milpitas High School,9.0,0.9,"Mixed, Covered",4.10 Acres,02201015,"$14,521,205","$175,211",95035
6,19800000,"7517 Deveron Ct,","$3,200",08/31/19,This lovely rental is located in the prestigio...,Apartment,1989,"Forced air, Gas",Central,"Garage, Garage - Attached, Covered",...,2.4,Silver Creek High School,8.0,4.1,"Garage, Garage - Attached, Covered",436 sqft,66557017,"$205,077","$3,264",95135
7,20635000,"3025 E 8th St,","$300,000",11/06/19,"3025 E 8th St, Los Angeles, CA 90023 is a sing...",SingleFamily,1922,Wall,Wall/Window Unit(s),"Garage, Covered",...,0.3,"Math, Science, & Technology Magnet Academy At ...",9.0,0.9,"Garage, Covered","2,953 sqft",5190008029,"$300,000","$3,786",90023


In [17]:
data.dtypes

Id                               int64
Address                         object
Sold Price                      object
Sold On                         object
Summary                         object
Type                            object
Year built                      object
Heating                         object
Cooling                         object
Parking                         object
Bedrooms                        object
Bathrooms                      float64
Total interior livable area     object
Total spaces                   float64
Garage spaces                  float64
Home type                       object
Region                          object
Elementary School               object
Elementary School Score        float64
Elementary School Distance     float64
High School                     object
High School Score              float64
High School Distance           float64
Parking features                object
Lot size                        object
Parcel number            

The types of many of the columns don't look right. You'd expect several of these to be numeric, but they are strings. So we will try to fix these features. But before that let's look at the most frequently occurring values in each of the columns to get an idea of what they contain.

In [18]:
# top 10 (most frequent) values in each column
[data[d].value_counts()[:10] for d in data.columns]

[20926300     1
 19608466     1
 19778666     1
 19766166     1
 15569866     1
 16164766     1
 25489466     1
 20763466     1
 19607966     1
 243210766    1
 Name: Id, dtype: int64,
 237 Bluebonnet Ln,        5
 480 E Okeefe St,          4
 1 Appian Way,             4
 1170 Foster City Blvd,    3
 401 S Norfolk St,         3
 129 Prospect Ct,          3
 88 S Broadway,            3
 1982 W Bayshore Rd,       3
 935 Old County Rd,        3
 404 Boardwalk Ave,        3
 Name: Address, dtype: int64,
 $1,100,000    661
 $1,200,000    652
 $1,300,000    564
 $1,250,000    529
 $1,500,000    476
 $1,000,000    474
 $1,400,000    469
 $1,150,000    467
 $1,050,000    466
 $850,000      465
 Name: Sold Price, dtype: int64,
 02/26/21    1163
 02/19/21     933
 01/29/21     825
 02/05/21     709
 02/18/21     702
 02/25/21     687
 02/17/21     685
 01/15/21     681
 02/24/21     655
 02/12/21     645
 Name: Sold On, dtype: int64,
 -                                                            

The above lists the 10 most frequent value for each of the feature. The feature value and fequencies are list, with the name of the feature at the end of the list. Some of the interesting features are: sold price, Type and Home type (what's the dfference?), Region, Zip.

Let's fix the currency (dollar) fields and the area fields

In [19]:
# convert to numeric
currency = ['Sold Price', 'Tax assessed value', 'Annual tax amount']
for c in currency:
    data[c] = data[c].replace(
        r'[$,-]', '', regex=True).replace(
        r'^\s*$', np.nan, regex=True).astype(float)

In [20]:
# convert area to sqft
#
areas = ['Total interior livable area', 'Lot size']
for c in areas:
    acres = data[c].str.contains('Acres') == True
    col = data[c].replace(r'\b sqft\b|\b Acres\b|\b,\b','', regex=True).astype(float)
    col[acres] *= 43560
    data[c] = col

In [21]:
# convert date features to date format
data['Sold On'] = pd.to_datetime(data['Sold On'], errors='coerce')
data['Year built'] = pd.to_numeric(data['Year built'], errors='coerce')

In [22]:
# convert bedrooms to numeric
data['Bedrooms'] = pd.to_numeric(data['Bedrooms'], errors='coerce')

In [23]:
data.isnull().sum()

Id                                 0
Address                            0
Sold Price                         0
Sold On                            0
Summary                            0
Type                               0
Year built                       200
Heating                            0
Cooling                            0
Parking                            0
Bedrooms                       12234
Bathrooms                          0
Total interior livable area        0
Total spaces                       0
Garage spaces                      0
Home type                          0
Region                             0
Elementary School                  0
Elementary School Score            0
Elementary School Distance         0
High School                        0
High School Score                  0
High School Distance               0
Parking features                   0
Lot size                           0
Parcel number                      0
Tax assessed value                 0
A

Some of the values could not be properly converted, because they were not properly formatted or were not numbers. This introduced some NaN values. We'll drop these.

In [24]:
data.dropna(inplace=True)

In [25]:
data.shape

(73286, 29)

In [26]:
# let's add two new features
# we will add two features: 1) age of house, directly related to year built and 2) price per sqft

data['Age'] = 2022 - data['Year built']

data['Price per living sqft'] = data['Sold Price'] / data['Total interior livable area']

All the sales data is from CA. We will assume (somewhat arbitrarily) that zip codes less than 93500 are southern California (SoCal) and the rest are northern California (NorCal). We will create a feature called `Region2` to capture this.

In [27]:
data['Region2'] = ''
data.loc[ data.Zip <= 93500, 'Region2' ] = 'SoCal'
data.loc[ data.Zip > 93500, 'Region2' ] = 'NorCal'

In [28]:
data.shape

(73286, 32)

In [29]:
data.columns

Index(['Id', 'Address', 'Sold Price', 'Sold On', 'Summary', 'Type',
       'Year built', 'Heating', 'Cooling', 'Parking', 'Bedrooms', 'Bathrooms',
       'Total interior livable area', 'Total spaces', 'Garage spaces',
       'Home type', 'Region', 'Elementary School', 'Elementary School Score',
       'Elementary School Distance', 'High School', 'High School Score',
       'High School Distance', 'Parking features', 'Lot size', 'Parcel number',
       'Tax assessed value', 'Annual tax amount', 'Zip', 'Age',
       'Price per living sqft', 'Region2'],
      dtype='object')

## Exploring the Data Set

### Problem 1
(10 points)

For each of these ten features: `Sold Price`, `Year built`,  `Heating`, `Bedrooms`, `Home type`, `High School Score`, `High School Distance`, `Zip`, `Age`, `Region2`, assign them one or more of the following eight tags: categorical, numeric, nominal, symmetric binary, asymmetric binary, ordinal, interval, and ratio.
  

**Solution:**




### Problem 2
(5 points)

Plot the distribution of the `Age` feature as a histogram. Do you see any obvious outliers? Remove the outliers and replot the histogram.

    
    

**Solution:**

### Problem 3
(5 points)

Plot a boxplot of `Age` vs `Region2`. How do the ages compare for the two values of `Region2`? 

**Solution:**

### Problem 4
(10 pints)

**Hypothesis:** If you one of High or Elementary schools is good at a location, then the other is good too. In other words, if a High school at a location has a high rating so would the elemntary school there and vice versa.

Using this data set, provide evidence for or against this hypothesis. Why is it harder to provide this evidence through visualization?

**Solution:**

### Problem 5
(20 points)

For NorCal region, find two Zip codes: 1) with the lowest median `Sold Price`; and, 2) with the highest median `Sold Price`. Both these Zip codes must have at least 20 data points. Plot a boxplot with data just from these two Zip codes, with the Zip codes on the x-axis and the `Sold Price` in log scale on the y-axis. Do you see any outliers?

Make sure the conditions that `Region2` is NorCal and that there are at least 20 data points in that Zip code are satisfied.

(Hint: use pandas groupby)  


**Solution:**